In [25]:
import os 
import pandas as pd 
import numpy as np 

In [31]:
## for TL data 
resdic = '/endosome/work/PCDC/s202005/Research/TransferLearning/Shelburne/Results/TL/Single_1_Anti_2_Dense_1024_Ensemble/Epochs_100_Batch_128/summary'
data_path = os.path.join(resdic,"test_summary_100_120.csv")
data = pd.read_csv(data_path,index_col=[0])
data.replace("LRPT","TL",inplace=True)
data.rename(columns={"LRPT":"config"},inplace=True)
data["method"] = "TL"

data["config_split"] = data.apply(lambda x:x["config"].split("_"),axis=1)
data["config_epochs"] = data.apply(lambda x:int(x["config_split"][x["config_split"].index("Epochs")+1]),axis=1)
data["config_batch"] =data.apply(lambda x:int(x["config_split"][x["config_split"].index("Batch")+1]),axis=1) 
data["config_pred_weights"] =data.apply(lambda x:x["config_split"][0],axis=1) 
data["config_input_weights"] = np.nan

data.drop(["config_split"],axis=1,inplace=True)
data.sort_values(["config_epochs","config_batch","config_input_weights","config_pred_weights"],inplace=True)

In [54]:
# for lrpt
resdic = '/endosome/work/PCDC/s202005/Research/TransferLearning/Shelburne/Results/LRPT/Single_1_Anti_2_Dense_1024_Ensemble/Archive/summary'
data_path = os.path.join(resdic,"test_summary.csv")
LRPT = pd.read_csv(data_path,index_col=[0])
LRPT = LRPT[LRPT.seed.isin(data.seed)]

LRPT.rename(columns={"LRPT":"config"},inplace=True)
LRPT["method"] = "LRPT"

LRPT["config_split"] = LRPT.apply(lambda x:x["config"].split("_"),axis=1)
LRPT["config_epochs"] = LRPT.apply(lambda x:int(x["config_split"][x["config_split"].index("Epochs")+1]),axis=1)
LRPT["config_batch"] =LRPT.apply(lambda x:int(x["config_split"][x["config_split"].index("Batch")+1]),axis=1) 
LRPT["config_pred_weights"] =LRPT.apply(lambda x:x["config_split"][0],axis=1) 
LRPT["config_input_weights"] = LRPT.apply(lambda x:x["config_split"][1],axis=1) 

LRPT.drop(["config_split"],axis=1,inplace=True)
LRPT.sort_values(["config_epochs","config_batch","config_input_weights","config_pred_weights"],inplace=True)
LRPT = LRPT[data.columns]

In [65]:
Total = pd.concat([LRPT,data])
save_path = "/endosome/work/PCDC/s202005/Research/TransferLearning/Shelburne/Results/summary/Single_1_Anti_2_Dense_1024_Ensemble"
os.makedirs(save_path,exist_ok=True)
Total.to_csv(os.path.join(save_path,"Epochs_100_Batch_128_total.csv"),index=False)

# for dashapp

In [19]:
Total_temp = Total[(Total.method == "LRPT") & (Total.config_batch==16)]
col_interest = "f1score"
agg_func = {col_interest:["describe"]}
xx = Total_temp.groupby(["config","model"]).agg(agg_func).reset_index()

In [41]:
x2 = xx.T.reset_index()
x2.drop(columns=["level_0",'level_1'],inplace=True)
x2.loc[0,"level_2"] = "config"
x2.loc[1,"level_2"] = "model"
x2 = x2.T
x2.columns = x2.iloc[0]
x2.drop(x2.index[0],inplace=True)

In [50]:
Total_temp = pd.merge(Total_temp,x2,how="left")

In [73]:
Total_temp["input_w"] = Total_temp["config_input_weights"]
Total_temp["pred_w"] = Total_temp["config_pred_weights"]
Total_temp["epochs"] = Total_temp["config_epochs"]
Total_temp["batch"] = Total_temp["config_batch"]

In [72]:
Total_temp.columns

Index(['model', 'threshold', 'f1score', 'precision_score', 'accuracy_score',
       'balanced_accuracy_score', 'recall_score', 'roc_auc_score', 'seed',
       'cv', 'config', 'method', 'config_epochs', 'config_batch',
       'config_input_weights', 'config_pred_weights', 'count', 'mean', 'std',
       'min', '25%', '50%', '75%', 'max', 'input_w', 'pred_w'],
      dtype='object')

In [76]:
import plotly.express as px 

fig = px.box(Total_temp,x="config",y=col_interest,color="model", 
hover_name="model",points="all",
                 hover_data={'config':False,"input_w":True,"pred_w":True,"epochs":True,"batch":True,"model":False,
                 col_interest:":.2f","count":True,
                             'mean':':.2f','std':":.2f",'25%':':.2f',
                             '50%':':.2f', '75%':':.2f'})
# #fig.update_traces(quartilemethod="exclusive")
fig.update_xaxes(title="",visible=True,showticklabels=True)
fig.update_yaxes(title="Prediction Accuracy with threshold=0.5")
fig.show()

In [127]:
Total

model  threshold   f1score  precision_score  accuracy_score  \
0   PARP        0.5  0.559242         0.433824        0.772059   
1   LRPT        0.5  0.685714         0.738462        0.892157   
0   PARP        0.5  0.482051         0.373016        0.752451   
1   LRPT        0.5  0.709677         0.800000        0.911765   
0   PARP        0.5  0.545455         0.434211        0.730392   
..   ...        ...       ...              ...             ...   
1     TL        0.5  0.222222         0.458333        0.811275   
0   PARP        0.5  0.444444         0.310345        0.669118   
1     TL        0.5  0.400000         0.470588        0.823529   
0   PARP        0.5  0.448399         0.329843        0.620098   
1     TL        0.5  0.340741         0.511111        0.781863   

    balanced_accuracy_score  recall_score  roc_auc_score      seed    cv  \
0                  0.777718      0.786667       0.837518  Seed_100  cv_0   
1                  0.794474      0.640000       0.923804  Seed_100  cv_0   
0                  0.724061      0.681159       0.790411  Seed_100  cv_1   
1                  0.802616      0.637681       0.954341  Seed_100  cv_1   
0                  0.731447      0.733333       0.791964  Seed_100  cv_2   
..                      ...           ...            ...       ...   ...   
1                  0.553814      0.146667       0.608348  Seed_106  cv_0   
0                  0.714313      0.782609       0.778077  Seed_106  cv_1   
1                  0.634090      0.347826       0.676542  Seed_106  cv_1   
0                  0.648742      0.700000       0.741247  Seed_106  cv_2   
1                  0.593187      0.255556       0.671855  Seed_106  cv_2   

                                            config method  config_epochs  \
0   CopiedWeights_CopiedWeights_Epochs_10_Batch_16   LRPT             10   
1   CopiedWeights_CopiedWeights_Epochs_10_Batch_16   LRPT             10   
0   CopiedWeights_CopiedWeights_Epochs_10_Batch_16   LRPT             10   
1   CopiedWeights_CopiedWeights_Epochs_10_Batch_16   LRPT             10   
0   CopiedWeights_CopiedWeights_Epochs_10_Batch_16   LRPT             10   
..                                             ...    ...            ...   
1              DefaultWeights_Epochs_100_Batch_128     TL            100   
0              DefaultWeights_Epochs_100_Batch_128     TL            100   
1              DefaultWeights_Epochs_100_Batch_128     TL            100   
0              DefaultWeights_Epochs_100_Batch_128     TL            100   
1              DefaultWeights_Epochs_100_Batch_128     TL            100   

    config_batch config_input_weights config_pred_weights  
0             16        CopiedWeights       CopiedWeights  
1             16        CopiedWeights       CopiedWeights  
0             16        CopiedWeights       CopiedWeights  
1             16        CopiedWeights       CopiedWeights  
0             16        CopiedWeights       CopiedWeights  
..           ...                  ...                 ...  
1            128                  NaN      DefaultWeights  
0            128                  NaN      DefaultWeights  
1            128                  NaN      DefaultWeights  
0            128                  NaN      DefaultWeights  
1            128                  NaN      DefaultWeights  

[20352 rows x 16 columns]

In [32]:
import plotly.express as px 
fig = px.box(summary_df,x="LRPT",y="roc_auc_score",color="model",points="all")
fig.update_traces(quartilemethod="exclusive")
fig.update_xaxes(title="LRPT Version",visible=True,showticklabels=False)
fig.update_yaxes(title="AUC")
fig.show()

In [72]:
Result

NameError: name 'Result' is not defined